In [1]:
import os
import math
import random

In [2]:
import json
import numpy as np
import mxnet as mx
import cv2

import matplotlib.pyplot as plt

In [3]:
def Conv(data, num_filter=1, kernel=(1, 1), stride=(1, 1), pad=(0, 0), num_group=1, name=None, suffix='', cudnn_off=False):
    conv = mx.sym.Convolution(data=data, num_filter=num_filter, kernel=kernel, num_group=num_group, stride=stride, pad=pad, no_bias=True, name='%s' % name, cudnn_off=cudnn_off)
    bn = mx.sym.BatchNorm(data=conv, name='%s_bn' % name, fix_gamma=False, use_global_stats=False, eps=0.0001, attr={'lr_mult': '1.0'})
    act = mx.sym.Activation(data=bn, act_type='relu', name='%s_relu' % name )
    return act

In [4]:
def mobile_net(w_factor=1.):
    data = mx.symbol.Variable(name='data')
    #label = mx.symbol.Variable(name="label")
    
    w1 = int(w_factor * 32) # 32
    w2 = w1 * 2 # 64
    w3 = w2 * 2 # 128
    w4 = w3 * 2 # 256
    w5 = w4 * 2 # 512
    w6 = w5 * 2 # 1024
    
    conv1 = Conv(data, num_filter=w1, kernel=(3, 3), pad=(1, 1), stride=(2, 2), name="conv1") # 224/112

    conv2_1_dw = Conv(conv1, num_group=w1, num_filter=w1, kernel=(3, 3), pad=(1, 1), stride=(1, 1), name="conv2_1_dw", cudnn_off=True) # 112/112
    conv2_1_sep = Conv(conv2_1_dw, num_filter=w2, kernel=(1, 1), pad=(0, 0), stride=(1, 1), name="conv2_1_sep") # 112/112
    conv2_2_dw = Conv(conv2_1_sep, num_group=w2, num_filter=w2, kernel=(3, 3), pad=(1, 1), stride=(2, 2), name="conv2_2_dw", cudnn_off=True) # 112/56
    conv2_2_sep = Conv(conv2_2_dw, num_filter=w3, kernel=(1, 1), pad=(0, 0), stride=(1, 1), name="conv2_2_sep") # 56/56

    conv3_1_dw = Conv(conv2_2_sep, num_group=w3, num_filter=w3, kernel=(3, 3), pad=(1, 1), stride=(1, 1), name="conv3_1_dw", cudnn_off=True) # 56/56
    conv3_1_sep = Conv(conv3_1_dw, num_filter=w3, kernel=(1, 1), pad=(0, 0), stride=(1, 1), name="conv3_1_sep") # 56/56
    conv3_2_dw = Conv(conv3_1_sep, num_group=w3, num_filter=w3, kernel=(3, 3), pad=(1, 1), stride=(2, 2), name="conv3_2_dw", cudnn_off=True) # 56/28
    conv3_2_sep = Conv(conv3_2_dw, num_filter=w4, kernel=(1, 1), pad=(0, 0), stride=(1, 1), name="conv3_2_sep") # 28/28

    conv4_1_dw = Conv(conv3_2_sep, num_group=w4, num_filter=w4, kernel=(3, 3), pad=(1, 1), stride=(1, 1), name="conv4_1_dw", cudnn_off=True) # 28/28
    conv4_1_sep = Conv(conv4_1_dw, num_filter=w4, kernel=(1, 1), pad=(0, 0), stride=(1, 1), name="conv4_1_sep") # 28/28
    conv4_2_dw = Conv(conv4_1_sep, num_group=w4, num_filter=w4, kernel=(3, 3), pad=(1, 1), stride=(2, 2), name="conv4_2_dw", cudnn_off=True) # 28/14
    conv4_2_sep = Conv(conv4_2_dw, num_filter=w5, kernel=(1, 1), pad=(0, 0), stride=(1, 1), name="conv4_2_sep") # 14/14

    conv5_1_dw = Conv(conv4_2_sep, num_group=w5, num_filter=w5, kernel=(3, 3), pad=(1, 1), stride=(1, 1), name="conv5_1_dw", cudnn_off=True) # 14/14
    conv5_1_sep = Conv(conv5_1_dw, num_filter=w5, kernel=(1, 1), pad=(0, 0), stride=(1, 1), name="conv5_1_sep") # 14/14
    conv5_2_dw = Conv(conv5_1_sep, num_group=w5, num_filter=w5, kernel=(3, 3), pad=(1, 1), stride=(1, 1), name="conv5_2_dw", cudnn_off=True) # 14/14
    conv5_2_sep = Conv(conv5_2_dw, num_filter=w5, kernel=(1, 1), pad=(0, 0), stride=(1, 1), name="conv5_2_sep") # 14/14
    conv5_3_dw = Conv(conv5_2_sep, num_group=w5, num_filter=w5, kernel=(3, 3), pad=(1, 1), stride=(1, 1), name="conv5_3_dw", cudnn_off=True) # 14/14
    conv5_3_sep = Conv(conv5_3_dw, num_filter=w5, kernel=(1, 1), pad=(0, 0), stride=(1, 1), name="conv5_3_sep") # 14/14
    conv5_4_dw = Conv(conv5_3_sep, num_group=w5, num_filter=w5, kernel=(3, 3), pad=(1, 1), stride=(1, 1), name="conv5_4_dw", cudnn_off=True) # 14/14
    conv5_4_sep = Conv(conv5_4_dw, num_filter=w5, kernel=(1, 1), pad=(0, 0), stride=(1, 1), name="conv5_4_sep") # 14/14
    conv5_5_dw = Conv(conv5_4_sep, num_group=w5, num_filter=w5, kernel=(3, 3), pad=(1, 1), stride=(1, 1), name="conv5_5_dw", cudnn_off=True) # 14/14
    conv5_5_sep = Conv(conv5_5_dw, num_filter=w5, kernel=(1, 1), pad=(0, 0), stride=(1, 1), name="conv5_5_sep") # 14/14
    conv5_6_dw = Conv(conv5_5_sep, num_group=w5, num_filter=w5, kernel=(3, 3), pad=(1, 1), stride=(2, 2), name="conv5_6_dw", cudnn_off=True) # 14/7
    conv5_6_sep = Conv(conv5_6_dw, num_filter=w6, kernel=(1, 1), pad=(0, 0), stride=(1, 1), name="conv5_6_sep") # 7/7

    conv6_dw = Conv(conv5_6_sep, num_group=w6, num_filter=w6, kernel=(3, 3), pad=(1, 1), stride=(1, 1), name="conv6_dw", cudnn_off=True) # 7/7
    conv6_sep = Conv(conv6_dw, num_filter=w6, kernel=(1, 1), pad=(0, 0), stride=(1, 1), name="conv6_sep") # 7/7

    pool6 = mx.symbol.Pooling(name='pool6', data=conv6_sep , pooling_convention='full', global_pool=True, kernel=(1,1), pool_type='avg')

    fc7 = mx.symbol.Convolution(name='fc7', data=pool6 , num_filter=2, pad=(0, 0), kernel=(1,1), stride=(1,1), no_bias=False)
    flatten = mx.symbol.Flatten(data=fc7, name='flatten')
    softmax = mx.symbol.SoftmaxOutput(data=flatten, name='softmax')
    return softmax
    

In [5]:
data_shape = 128
mean_val = np.array([[[123.68]],[[116.78]],[[103.94]]])
std_scale = 0.017

In [6]:
def get_dg_db():
    dat_path = '/mnt/6B133E147DED759E/Archive2017.8.16_res'

    with open(dat_path + '/data.json') as f:
        infos = json.load(f)

    #imgs = [cv2.cvtColor(cv2.imread(dat_path+'/'+i['filename']),cv2.COLOR_BGR2RGB) for i in infos]
    #imgs = [cv2.cvtColor(cv2.imread(dat_path+'/'+i['filename']),cv2.COLOR_BGR2RGB) for i in infos]
    imgs = []
    for i in infos:
        img = cv2.imread(dat_path+'/'+i['filename'])
        #print(i['filename'])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         x0 = i['xmin']
#         x1 = i['xmax']
#         y0 = i['ymin']
#         y1 = i['ymax']
#         cv2.rectangle(img, (x0,y0),(x1,y1),(0,0,255),3)
#         plt.imshow(img)
#         plt.show()
        imgs.append(img)
    
    return infos, imgs

In [7]:
class DGIter(mx.io.DataIter):
    def __init__(self, db, batch_size=1, image_shape=128, data_name='data', label_name='softmax_label'):
        super(DGIter, self).__init__(batch_size)
        
        self.db_json = db[0]
        #self.db_dict = db[0]
        #self.imgfiles = db[2]
        self.imgs = db[1]
        #self.masks = db[4]
        self.image_shape = image_shape
        
        self.num_data = len(self.imgs)
        self.idx = np.arange(0, self.num_data)
        np.random.shuffle(self.idx)
        
        assert self.num_data >= batch_size, "batch_size needs to be smaller than data size."
        
        self.cursor = -batch_size
        self.batch_size = batch_size
    
    # TODO: 
    @property
    def provide_data(self):
        """The name and shape of data provided by this iterator."""
        return [
            mx.io.DataDesc('data', (self.batch_size, 3, self.image_shape, self.image_shape))
        ]

    # TODO: 
    @property
    def provide_label(self):
        """The name and shape of label provided by this iterator."""
        return [
             mx.io.DataDesc('y', (self.batch_size,)) #, mx.io.DataDesc('kps', (self.batch_size, 10))
        ]
    
    def hard_reset(self):
        np.random.shuffle(self.idx)
        """Ignore roll over data and set to start."""
        self.cursor = -self.batch_size
        
    def reset(self):
        self.cursor = -self.batch_size
        
    def iter_next(self):
        self.cursor += self.batch_size
        return self.cursor < self.num_data

    def sample_batch(self, sample_list):
        imgs = np.empty(shape=(self.batch_size, 3, self.image_shape, self.image_shape), dtype=np.float32)
        y = np.empty(shape=(self.batch_size,), dtype=np.float32)

        for k,i in enumerate(sample_list):
            img = self.imgs[i]
            info = self.db_json[i]
            
            x0 = info['xmin']
            x1 = info['xmax']
            y0 = info['ymin']
            y1 = info['ymax']
            
            dg_type = info['type']

            dst_w = self.image_shape
            dst_h = self.image_shape
            obj_w = x1 - x0
            obj_h = y1 - y0
            xc = (x0 + x1) / 2.
            yc = (y0 + y1) / 2.
            #scale = dst_w / max(obj_w, obj_h)
            scale_w = dst_w / float(obj_w)
            scale_h = dst_h / float(obj_h)

            sj = random.uniform(0.9, 1.2)
            scale_jitter_w = sj * random.choice([-1,1])
            scale_jitter_h = sj * random.choice([-1,1])
            
            scale_w *= scale_jitter_w
            scale_h *= scale_jitter_h
            
            dst_w_jitter = random.uniform(-20,20)
            dst_h_jitter = random.uniform(-15,15)
            theta = random.uniform(-np.pi/40, np.pi/40)

            T0 = np.array([[1,0,-xc],[0,1,-yc],[0,0,1]])
            S = np.array([[scale_w,0,0],[0, scale_h,0],[0,0,1]])
            R = np.array([[np.cos(theta), np.sin(theta), 0], [-np.sin(theta), np.cos(theta), 0],[0,0,1]])
            T1 = np.array([[1,0,dst_w/2. +dst_w_jitter],[0,1,dst_h/2. +dst_h_jitter],[0,0,1]])
            M = np.dot(S, T0)
            M = np.dot(R, M)
            M = np.dot(T1, M)
            M_warp = M[0:2,:]

            dst_img = cv2.warpAffine(img, M_warp, dsize=(int(dst_w), int(dst_h)))
            dst_img = np.transpose(dst_img,(2,0,1))
            dst_img = std_scale * (dst_img.astype(np.float32) - mean_val)
            
            imgs[k][:] = dst_img
            y[k] = dg_type
        return imgs, y
    
    def getdata(self):
        assert(self.cursor < self.num_data), "DataIter needs reset."
        if self.cursor + self.batch_size <= self.num_data:
            l = list(range(self.cursor, self.cursor + self.batch_size))
        else:
            pad = self.batch_size - self.num_data + self.cursor
            l = list(range(self.cursor, self.num_data)) + list(range(0,pad))
        
        
        
        sample_list = [self.idx[i] for i in l]
        images, y = self.sample_batch(sample_list)
        images = mx.nd.array(images)
        y = mx.nd.array(y)
        #kps = mx.nd.array(kps)
        return images, y

    #def getlabel(self):
    #    return self.rand_trans(self.label, True)
    
    # TODO: 
    def next(self):
        if self.iter_next():
            images, y = self.getdata()
            return mx.io.DataBatch(data=[images], label=[y], pad=self.getpad(), index=None)
        else:
            raise StopIteration
    
    def getpad(self):
        if self.cursor + self.batch_size > self.num_data:
            return self.cursor + self.batch_size - self.num_data
        else:
            return 0

In [8]:
db = get_dg_db()

In [9]:
ylb_iter = DGIter(db, 256, image_shape=data_shape)

In [10]:
# import matplotlib.pyplot as plt
# for i in range(20):
#     ylb_iter.reset()
#     for batch in ylb_iter:
#         plt.imshow(batch.data[0][0,1].asnumpy())
#         plt.show()
#         print(batch.label[0][0])
        

In [11]:
# mx.viz.plot_network(softmax,shape={'data':(8,1,224,224)}, node_attrs={'shape':'oval','fixedsize':'fasl==false'}).view()

In [12]:
pretrain=0

In [13]:
if pretrain:
    sym = mobile_net()
    layers = sym.get_internals()
    y = mx.sym.Variable('y')
    pool6 = layers['pool6_output']
    fc7 = mx.symbol.Convolution(name='fc7', data=pool6 , num_filter=2, pad=(0, 0), kernel=(1,1), stride=(1,1), no_bias=False)
    flatten = mx.symbol.Flatten(data=fc7, name='flatten')
    softmax = mx.sym.SoftmaxOutput(data=flatten, label=y, name = 'softmax')

    #mx.viz.plot_network(softmax,shape={'data':(8,3,data_shape,data_shape)}, node_attrs={'shape':'oval','fixedsize':'fasl==false'}).view()
    params = mx.nd.load('mobilenet-0000.params')
    graph=json.loads(sym.tojson())

    arg_params = {}
    aux_params = {}
    for i, n in enumerate(graph['nodes']):
        if n['op'] == 'Convolution' or n['op'] == 'FullyConnected':
            arg_params[n['name']+'_weight'] = params['arg:'+n['name']+'_weight']
            if n['attr']['no_bias'] == 'False':
                arg_params[n['name']+'_bias'] = params['arg:'+n['name']+'_bias']
        elif n['op'] == 'BatchNorm':
            arg_params[n['name']+'_gamma'] = params['arg:'+n['name']+'_gamma']
            arg_params[n['name']+'_beta'] = params['arg:'+n['name']+'_beta']
            aux_params[n['name']+'_moving_mean'] = params['aux:'+n['name']+'_moving_mean']
            aux_params[n['name']+'_moving_var'] = params['aux:'+n['name']+'_moving_var']
        else:
            pass
    
    arg_shapes, _, aux_shapes = softmax.infer_shape(data=(1,3,data_shape,data_shape))
    arg_names = softmax.list_arguments()
    aux_names = softmax.list_auxiliary_states()
    arg_shape_dic = dict(zip(arg_names, arg_shapes))
    aux_shape_dic = dict(zip(aux_names, aux_shapes))

    arg_params['fc7_weight'] = mx.nd.random_normal(shape=(arg_shape_dic['fc7_weight'])) *.01
    arg_params['fc7_bias'] = mx.nd.zeros(shape=(arg_shape_dic['fc7_bias'])) *.01
else:   
    sym = mobile_net(0.25)
    layers = sym.get_internals()
    y = mx.sym.Variable('y')
    pool6 = layers['pool6_output']
    fc7 = mx.symbol.Convolution(name='fc7', data=pool6 , num_filter=2, pad=(0, 0), kernel=(1,1), stride=(1,1), no_bias=False)
    flatten = mx.symbol.Flatten(data=fc7, name='flatten')
    softmax = mx.sym.SoftmaxOutput(data=flatten, label=y, name = 'softmax')

In [14]:
ctx = [mx.gpu(0),mx.gpu(1),mx.gpu(2),mx.gpu(3)]
mod = mx.mod.Module(softmax, data_names=['data'], label_names=['y'], context=ctx)

In [15]:
#mx.viz.plot_network(softmax,shape={'data':(8,3,data_shape,data_shape)}, node_attrs={'shape':'oval','fixedsize':'fasl==false'}).view()

In [16]:
mod.bind(for_training=True, data_shapes=ylb_iter.provide_data, label_shapes=ylb_iter.provide_label)
if pretrain:
    print("PRETRAIN")
    mod.set_params(arg_params=arg_params, aux_params=aux_params, allow_extra=True)
else:
    print("SCRATCH")
    mod.init_params(initializer=mx.init.Xavier(magnitude=2.))

SCRATCH


In [17]:
import logging
logging.basicConfig(level=logging.INFO)

In [19]:
batches = ylb_iter.num_data / float(ylb_iter.batch_size)
lr_sch = mx.lr_scheduler.MultiFactorScheduler([int(32 * batches), int(64 * batches)], 0.5)
#optimizer =mx.optimizer.SGD(momentum = 0.9, learning_rate=0.0001, momentum=0.9, lr_scheduler=lr_sch, rescale_grad=1.0/nd_iter.batch_size)
mod.init_optimizer(optimizer='RMSProp', optimizer_params={'learning_rate':0.001, 'lr_scheduler': lr_sch, 'wd': 0.00005,
                                                         'rescale_grad': 1.0 / len(ctx) if len(ctx) > 0 else 1.0 })
#mod.init_optimizer(optimizer='sgd', optimizer_params={'learning_rate':0.001, 'momentum': 0.9, 'lr_scheduler': lr_sch})

In [20]:
epoch_end_callback = mx.callback.do_checkpoint("dg")
ylb_iter.reset()
mod.fit(ylb_iter, num_epoch=100,eval_metric= 'acc', epoch_end_callback = epoch_end_callback)
#metric = mx.metric.create('acc')


/home/dingkou/dev/incubator-mxnet/python/mxnet/module/base_module.py:464: UserWarning: Parameters already initialized and force_init=False. init_params call ignored.
  allow_missing=allow_missing, force_init=force_init)
INFO:root:Epoch[0] Train-accuracy=0.839323
INFO:root:Epoch[0] Time cost=7.142
INFO:root:Saved checkpoint to "dg-0001.params"
INFO:root:Epoch[1] Train-accuracy=0.894618
INFO:root:Epoch[1] Time cost=6.617
INFO:root:Saved checkpoint to "dg-0002.params"
INFO:root:Epoch[2] Train-accuracy=0.914497
INFO:root:Epoch[2] Time cost=6.505
INFO:root:Saved checkpoint to "dg-0003.params"
INFO:root:Epoch[3] Train-accuracy=0.922309
INFO:root:Epoch[3] Time cost=6.497
INFO:root:Saved checkpoint to "dg-0004.params"
INFO:root:Epoch[4] Train-accuracy=0.941146
INFO:root:Epoch[4] Time cost=6.645
INFO:root:Saved checkpoint to "dg-0005.params"
INFO:root:Epoch[5] Train-accuracy=0.947309
INFO:root:Epoch[5] Time cost=6.548
INFO:root:Saved checkpoint to "dg-0006.params"
INFO:root:Epoch[6] Train-accur

INFO:root:Update[2832]: Change learning rate to 2.50000e-04
INFO:root:Epoch[62] Train-accuracy=0.996701
INFO:root:Epoch[62] Time cost=6.500
INFO:root:Saved checkpoint to "dg-0063.params"
INFO:root:Epoch[63] Train-accuracy=0.997830
INFO:root:Epoch[63] Time cost=6.558
INFO:root:Saved checkpoint to "dg-0064.params"
INFO:root:Epoch[64] Train-accuracy=0.996875
INFO:root:Epoch[64] Time cost=6.349
INFO:root:Saved checkpoint to "dg-0065.params"
INFO:root:Epoch[65] Train-accuracy=0.997222
INFO:root:Epoch[65] Time cost=6.439
INFO:root:Saved checkpoint to "dg-0066.params"
INFO:root:Epoch[66] Train-accuracy=0.997049
INFO:root:Epoch[66] Time cost=6.342
INFO:root:Saved checkpoint to "dg-0067.params"
INFO:root:Epoch[67] Train-accuracy=0.997569
INFO:root:Epoch[67] Time cost=6.466
INFO:root:Saved checkpoint to "dg-0068.params"
INFO:root:Epoch[68] Train-accuracy=0.997135
INFO:root:Epoch[68] Time cost=6.528
INFO:root:Saved checkpoint to "dg-0069.params"
INFO:root:Epoch[69] Train-accuracy=0.997483
INFO:ro

In [19]:
metric = mx.metric.create('loss')
for epoch in range(90):
    ylb_iter.reset()
    metric.reset()
    for batch in ylb_iter:
        mod.forward(batch, is_train=True)       # compute predictions
        
        mod.update_metric(metric, batch.label)  # accumulate prediction accuracy
        mod.backward()                          # compute gradients
        mod.update()                            # update parameters
    print('Epoch %d, Training %s' % (epoch, metric.get()))

Epoch 0, Training ('loss', 0.49999999735090467)
Epoch 1, Training ('loss', 0.49999999933772615)
Epoch 2, Training ('loss', 0.49999999867545236)


KeyboardInterrupt: 

In [ ]:
mod.save_checkpoint('ylb', 199)